# 수원 도시정책지표 분석 노트북 (Windows/Jupyter)

아래 셀을 순서대로 실행하십시오. 입력 파일 경로만 맞추면 나머지는 자동으로 처리됩니다.

In [18]:
# 0) 설정
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path.cwd().parent
print(ROOT)

# Windows 경로 예시: r"D:\\workspace\\data\\수원시 도시정책지표 공개 데이터.xlsx"
INPUT_XLSX = Path(r"D:\workspace\dacon_sri\data\internal\3. 도시정책지표\수원시 도시정책지표 공개 데이터.xlsx")  # 변경 가능
SHEET_NAME = "수원시"  # 변경 가능
OUT_DIR = ROOT / "output" / "3. 도시정책지표"  # 변경 가능
OUT_DIR.mkdir(parents=True, exist_ok=True)
INPUT_XLSX, SHEET_NAME, OUT_DIR

d:\workspace\dacon_sri


(WindowsPath('D:/workspace/dacon_sri/data/internal/3. 도시정책지표/수원시 도시정책지표 공개 데이터.xlsx'),
 '수원시',
 WindowsPath('d:/workspace/dacon_sri/output/3. 도시정책지표'))

## 1) 적재 & 롱포맷 변환

In [19]:
df = pd.read_excel(INPUT_XLSX, sheet_name=SHEET_NAME, header=2)
df.columns = [str(c).replace("\n"," ").strip() for c in df.columns]

year_cols = [c for c in df.columns if str(c).isdigit() and 1990 <= int(str(c)) <= 2030]
year_cols = sorted(year_cols, key=lambda x: int(x))
meta_cols = [c for c in ["공개번호","지표명","산출방법","단위","출처"] if c in df.columns]

long_df = df.melt(id_vars=meta_cols, value_vars=year_cols, var_name="year", value_name="value")
long_df["year"] = pd.to_numeric(long_df["year"], errors="coerce").astype("Int64")
long_df["value"] = pd.to_numeric(long_df["value"], errors="coerce")
long_df.to_csv(OUT_DIR / "long.csv", index=False, encoding="utf-8-sig")
long_df.head()

,지표명,산출방법,단위,출처,year,value
0,총인구,NaN,NaN,NaN,2010,NaN
1,주민등록 기준,"주민등록인구 기준 총인구 수\n▶2010년 주민등록에 의한 집계(연말기준), 외국인...",명,"행정안전부, 「주민등록인구현황」",2010,1.077535e+06
2,인구총조사 기준,인구총조사 기준 총인구 수\n▶2015년-2023년 외국인 포함\n▶2010년 외국...,명,"통계청, 「인구총조사」",2010,1.054053e+06
3,청년인구 비율,주민등록 전체 인구 대비 청년(19~34세) 인구 수와 비율\n※ 청년인구비율 = ...,"명, %","행정안전부, 「주민등록인구현황」",2010,2.527417e+01
4,청년인구,NaN,NaN,NaN,2010,2.723380e+05


## 2) 품질 진단: 결측률/이상치

In [20]:
key = "공개번호" if "공개번호" in long_df.columns else (meta_cols[0] if meta_cols else "지표명")

# 연도별 결측률
miss_by_year = (
    long_df.assign(is_valid=long_df["value"].notna())
           .groupby("year")["is_valid"].mean()
           .rename("valid_ratio").reset_index()
)
miss_by_year["missing_ratio"] = 1 - miss_by_year["valid_ratio"]
miss_by_year["missing_ratio(%)"] = (miss_by_year["missing_ratio"]*100).round(1)
miss_by_year.to_csv(OUT_DIR / "quality_missing_by_year.csv", index=False, encoding="utf-8-sig")
miss_by_year.head()

,year,valid_ratio,missing_ratio,missing_ratio(%)
0,2010,0.31875,0.68125,68.1
1,2011,0.27500,0.72500,72.5
2,2012,0.28750,0.71250,71.2
3,2013,0.29375,0.70625,70.6
4,2014,0.35000,0.65000,65.0


In [21]:
# 그룹 키 안전하게 구성
base_keys = [key, "지표명", "단위", "출처"]
# 1) 순서 유지한 중복 제거  2) 실제 존재하는 컬럼만 사용
group_keys = [c for c in dict.fromkeys(base_keys) if c in long_df.columns]

# 지표별 결측률
miss_by_indicator = (
    long_df.assign(is_valid=long_df["value"].notna())
           .groupby(group_keys, dropna=False)["is_valid"]
           .mean()
           .to_frame("valid_ratio")      # Series → DataFrame
           .reset_index()
)

miss_by_indicator["missing_ratio"] = 1 - miss_by_indicator["valid_ratio"]
miss_by_indicator.sort_values("missing_ratio", ascending=False) \
    .to_csv(OUT_DIR / "quality_missing_by_indicator.csv", index=False, encoding="utf-8-sig")
miss_by_indicator.head()


,지표명,단위,출처,valid_ratio,missing_ratio
0,1인가구 비율,"가구, %","통계청, 「인구총조사」",0.666667,0.333333
1,1인가구 수,NaN,NaN,0.666667,0.333333
2,1인당 공원 면적,㎡/인,"▶공원: 경기도 수원시, 「수원기본통계」(자료: 경기도 정원산업과)\n▶도시인구: ...",0.666667,0.333333
3,1인당 문화관련 예산액,천원,▶문화관련 예산: 지방재정365(지방재정통합공개시스템)\n▶주민등록인구: 행정안전부...,1.000000,0.000000
4,1인당 생활폐기물 발생량,kg/일,환경부(폐자원관리과)\n,0.933333,0.066667


In [22]:
# z-score>3 이상치 플래그 (지표별)
def zflag(group):
    v = group["value"]
    m = v.mean(skipna=True)
    s = v.std(skipna=True, ddof=1)
    if pd.isna(s) or s == 0:
        group["outlier_flag"] = False
    else:
        group["outlier_flag"] = ((v - m).abs() > 3*s)
    return group

zlong = long_df.groupby([key], dropna=False, group_keys=False).apply(zflag)
outliers = zlong[zlong["outlier_flag"]==True]
outliers.to_csv(OUT_DIR / "outliers.csv", index=False, encoding="utf-8-sig")
outliers.head()

C:\Users\wsm\AppData\Local\Temp\ipykernel_3020\4269270746.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zlong = long_df.groupby([key], dropna=False, group_keys=False).apply(zflag)


,지표명,산출방법,단위,출처,year,value,outlier_flag


## 3) 단위/척도 정규화(%/퍼센트/율 자동 스케일)

In [23]:
norm = long_df.copy()
unit_series = norm["단위"].astype(str)

def needs_pct_scaling(sub):
    vals = sub["value"].dropna()
    if len(vals)==0: return False
    q95 = vals.quantile(0.95)
    return (q95<=1.2)

pct_mask = unit_series.str.contains("%|퍼센트|율", na=False)
for gid, sub in norm[pct_mask].groupby(key):
    if needs_pct_scaling(sub):
        idx = sub.index
        norm.loc[idx, "value_norm"] = sub["value"]*100

norm["value_norm"] = norm.get("value_norm", norm["value"])  # 없는 경우 원값 유지
norm.to_csv(OUT_DIR / "long_normalized.csv", index=False, encoding="utf-8-sig")
norm.head()

,지표명,산출방법,단위,출처,year,value,value_norm
0,총인구,NaN,NaN,NaN,2010,NaN,NaN
1,주민등록 기준,"주민등록인구 기준 총인구 수\n▶2010년 주민등록에 의한 집계(연말기준), 외국인...",명,"행정안전부, 「주민등록인구현황」",2010,1.077535e+06,1.077535e+06
2,인구총조사 기준,인구총조사 기준 총인구 수\n▶2015년-2023년 외국인 포함\n▶2010년 외국...,명,"통계청, 「인구총조사」",2010,1.054053e+06,1.054053e+06
3,청년인구 비율,주민등록 전체 인구 대비 청년(19~34세) 인구 수와 비율\n※ 청년인구비율 = ...,"명, %","행정안전부, 「주민등록인구현황」",2010,2.527417e+01,2.527417e+01
4,청년인구,NaN,NaN,NaN,2010,2.723380e+05,2.723380e+05


## 4) 추세 통계(연평균 기울기, CAGR)

In [25]:
# 0) 사전 정리
norm = norm.copy()
# year 숫자화 보강
norm["year"] = pd.to_numeric(norm["year"], errors="coerce")
# value_norm 없으면 원값 사용
if "value_norm" not in norm.columns:
    norm["value_norm"] = norm["value"]

# (선택) 컬럼명 중복 제거 — 첫 번째만 유지
if norm.columns.duplicated().any():
    norm = norm.loc[:, ~norm.columns.duplicated(keep="first")]

# 1) 인덱스 키 안전 구성(중복 제거 + 실제 존재 컬럼만)
base_keys = [key, "지표명", "단위", "출처"]
index_keys = [c for c in dict.fromkeys(base_keys) if c in norm.columns]

# 2) 피벗
wide = norm.pivot_table(
    index=index_keys,
    columns="year",
    values="value_norm",
    aggfunc="mean"
).sort_index(axis=1)

# 3) 연도 축/계산
years = [c for c in wide.columns if pd.notna(c)]
x = np.array(years, dtype=float)

def slope_and_cagr(row):
    y = row.values.astype(float)
    mask = ~np.isnan(y)
    out = {"slope_per_year": np.nan, "cagr": np.nan, "first_year": np.nan, "last_year": np.nan}
    if mask.sum() >= 2:
        coef = np.polyfit(x[mask], y[mask], 1)
        out["slope_per_year"] = float(coef[0])
        first, last = y[mask][0], y[mask][-1]
        n = mask.sum() - 1
        if n > 0 and first > 0 and last > 0:
            out["cagr"] = (last/first)**(1/n) - 1
        out["first_year"] = years[np.where(mask)[0][0]]
        out["last_year"]  = years[np.where(mask)[0][-1]]
    return pd.Series(out)

trends = wide.apply(slope_and_cagr, axis=1).reset_index()
trends["cagr(%)"] = (trends["cagr"] * 100).round(2)
trends.to_csv(OUT_DIR / "trends.csv", index=False, encoding="utf-8-sig")
trends.head()


,지표명,단위,출처,slope_per_year,cagr,first_year,last_year,cagr(%)
0,1인가구 비율,"가구, %","통계청, 「인구총조사」",0.878072,0.040943,2010.0,2023.0,4.09
1,1인당 공원 면적,㎡/인,"▶공원: 경기도 수원시, 「수원기본통계」(자료: 경기도 정원산업과)\n▶도시인구: ...",0.116337,0.008175,2013.0,2022.0,0.82
2,1인당 문화관련 예산액,천원,▶문화관련 예산: 지방재정365(지방재정통합공개시스템)\n▶주민등록인구: 행정안전부...,1.218868,0.042574,2010.0,2024.0,4.26
3,1인당 생활폐기물 발생량,kg/일,환경부(폐자원관리과)\n,0.004176,-0.008072,2010.0,2023.0,-0.81
4,1인당 온실가스 배출량,톤CO2eq/인,"경기도 수원시, 기후변화대책",-0.030000,-0.003759,2018.0,2021.0,-0.38


In [26]:
print("key =", key)
print("중복 컬럼 존재 여부:", norm.columns.duplicated().any())
print("피벗 인덱스 키:", index_keys)


key = 지표명
중복 컬럼 존재 여부: False
피벗 인덱스 키: ['지표명', '단위', '출처']


## 5) 상관/간이 클러스터(|r|≥0.7 연결요소)

In [29]:
# 상관행렬까지는 동일
valid_cols = [c for c in wide.columns if pd.notna(c)]
coverage = wide[valid_cols].notna().mean(axis=1)
wide_valid = wide.loc[coverage >= 0.7]

# 관측치 부족 시 빈 처리 방지
if wide_valid.shape[0] == 0:
    corr = pd.DataFrame()
else:
    corr = wide_valid.transpose().corr(method="pearson", min_periods=6)

corr.to_csv(OUT_DIR / "correlation_pearson.csv", encoding="utf-8-sig")

# corr가 비어있으면 이후 스킵
if corr.empty:
    clust_df = pd.DataFrame(columns=["group_id","공개번호","지표명","단위","출처","size"])
    clust_df.to_csv(OUT_DIR / "clusters_corr0p7.csv", index=False, encoding="utf-8-sig")
else:
    thr = 0.7
    idxs = list(corr.index)

    # 인덱스 레벨 이름(=wide_valid.index.names)을 사용
    index_names = list(wide_valid.index.names)
    # 공개번호 필드가 없을 수도 있으므로 fallback 지정
    id_field = "공개번호" if "공개번호" in index_names else index_names[0]

    # 인접리스트
    adj = {i: set() for i in idxs}
    for a in idxs:
        # NaN 상관 제거
        row = corr.loc[a]
        for b, r in row.items():
            if a != b and pd.notna(r) and abs(r) >= thr:
                adj[a].add(b)

    # 연결요소
    visited = set()
    groups = []
    for node in idxs:
        if node in visited:
            continue
        stack = [node]
        comp = []
        while stack:
            v = stack.pop()
            if v in visited:
                continue
            visited.add(v)
            comp.append(v)
            stack.extend([u for u in adj[v] if u not in visited])
        groups.append(comp)

    # 노드 → 메타 행 생성(레벨 수 가변 대응)
    clust_rows = []
    for gi, comp in enumerate(groups, start=1):
        size = len(comp)
        for node in comp:
            # node가 튜플이면 MultiIndex, 아니면 단일 인덱스
            if isinstance(node, tuple):
                meta = dict(zip(index_names, node))
            else:
                meta = {index_names[0]: node}

            clust_rows.append({
                "group_id": gi,
                "공개번호": meta.get("공개번호", meta.get(id_field, None)),
                "지표명":   meta.get("지표명", None),
                "단위":     meta.get("단위", None),
                "출처":     meta.get("출처", None),
                "size":     size,
            })

    clust_df = pd.DataFrame(clust_rows).sort_values(["group_id", "size"], ascending=[True, False])
    clust_df.to_csv(OUT_DIR / "clusters_corr0p7.csv", index=False, encoding="utf-8-sig")

clust_df.head()


,group_id,공개번호,지표명,단위,출처,size
0,1,1인당 문화관련 예산액,1인당 문화관련 예산액,천원,▶문화관련 예산: 지방재정365(지방재정통합공개시스템)\n▶주민등록인구: 행정안전부...,35
1,1,자연재해 피해액,자연재해 피해액,천원,"국민재난안전포털, 자연재난상황통계",35
2,1,외국인 총인구,외국인 총인구,명,"경기도 수원시, 「수원기본통계」",35
3,1,인구십만명당 문화기반시설수,인구십만명당 문화기반시설수,개,문화체육관광부(문화기반과),35
4,1,인구 십만명당 체육시설수,인구 십만명당 체육시설수,개,"경기도 수원시, 「수원기본통계」\n행정안전부, 「주민등록인구현황」",35


## 6) KPI 후보(결측≤20%, ≥8년, 기울기 상위 30%)

In [30]:
miss_rate = long_df.groupby([key], dropna=False)["value"].apply(lambda s: s.isna().mean()).reset_index(name="missing_ratio")
n_years = long_df.dropna(subset=["value"]).groupby([key])["year"].nunique().reset_index(name="n_years")
trend_strength = trends[[key,"slope_per_year"]].copy()
q70 = trend_strength["slope_per_year"].abs().quantile(0.70)

kpi = (trends
       .merge(miss_rate, on=key, how="left")
       .merge(n_years, on=key, how="left"))
kpi = kpi[(kpi["missing_ratio"]<=0.2) & (kpi["n_years"]>=8) & (kpi["slope_per_year"].abs()>=q70)]
kpi = kpi.sort_values("slope_per_year", ascending=False)
kpi.to_csv(OUT_DIR / "kpi_candidates.csv", index=False, encoding="utf-8-sig")
kpi.head()

,지표명,단위,출처,slope_per_year,cagr,first_year,last_year,cagr(%),missing_ratio,n_years
80,의료서비스접근성,건,"경기도, 「경기도기본통계」",304163.401099,0.013128,2010.0,2022.0,1.31,0.133333,13
82,이전재원,백만원,지방재정365(지방재정통합공개시스템),92081.560714,0.095646,2010.0,2024.0,9.56,0.000000,15
94,자체재원,백만원,지방재정365(지방재정통합공개시스템),77175.782143,0.059098,2010.0,2024.0,5.91,0.000000,15
70,아파트 수,호,"경기도 수원시, 「수원기본통계」",7875.382418,0.036882,2010.0,2023.0,3.69,0.066667,14
104,주민등록 기준,명,"행정안전부, 「주민등록인구현황」",7210.760714,0.007298,2010.0,2024.0,0.73,0.000000,15


## 산출물 안내
- `long.csv`, `long_normalized.csv`
- `quality_missing_by_year.csv`, `quality_missing_by_indicator.csv`, `outliers.csv`
- `trends.csv`, `correlation_pearson.csv`, `clusters_corr0p7.csv`, `kpi_candidates.csv`

In [43]:
# === Visualization Utils (Windows Jupyter) ====================================
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

FIG_DIR = Path(OUT_DIR) / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

# 한글 폰트(윈도우: 맑은 고딕) 설정(없으면 기본 폰트로 자동 대체)
try:
    matplotlib.rcParams["font.family"] = "Malgun Gothic"
except Exception:
    pass
matplotlib.rcParams["axes.unicode_minus"] = False

def _save(fig, name: str):
    p = FIG_DIR / f"{name}.png"
    fig.tight_layout()
    fig.savefig(p, dpi=160, bbox_inches="tight")
    plt.close(fig)
    print(f"[saved] {p}")

# 1) 연도별 결측률 막대
def plot_missing_by_year(csv_path=OUT_DIR / "quality_missing_by_year.csv"):
    df = pd.read_csv(csv_path)
    df = df.sort_values("year")
    fig, ax = plt.subplots(figsize=(10,4))
    ax.bar(df["year"].astype(str), df["missing_ratio"]*100)
    ax.set_title("연도별 결측률(%)")
    ax.set_xlabel("연도")
    ax.set_ylabel("결측률(%)")
    for i,(x,y) in enumerate(zip(df["year"].astype(str), df["missing_ratio"]*100)):
        if y>0:
            ax.text(i, y, f"{y:.1f}", ha="center", va="bottom", fontsize=8, rotation=0)
    _save(fig, "01_missing_by_year")

# 2) 지표별 결측률 Top-N (가로 막대)
def plot_missing_by_indicator_topn(csv_path=OUT_DIR / "quality_missing_by_indicator.csv", topn=20):
    df = pd.read_csv(csv_path)
    df = df.sort_values("missing_ratio", ascending=False).head(topn)
    fig, ax = plt.subplots(figsize=(10, max(4, 0.35*len(df))))
    ax.barh(df["지표명"], df["missing_ratio"]*100)
    ax.set_title(f"지표별 결측률 Top {topn} (단위: %)")
    ax.set_xlabel("결측률(%)")
    ax.invert_yaxis()
    for y,(name,val) in enumerate(zip(df["지표명"], df["missing_ratio"]*100)):
        ax.text(val, y, f"{val:.1f}%", va="center", ha="left", fontsize=8)
    _save(fig, "02_missing_by_indicator_topN")

# 3) 추세 산점도: slope vs CAGR
def plot_trends_scatter(csv_path=OUT_DIR / "trends.csv", label_top=10):
    df = pd.read_csv(csv_path)
    # 결측 제거
    df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=["slope_per_year"])
    df["cagr_pct"] = df["cagr"].fillna(0)*100
    fig, ax = plt.subplots(figsize=(8,6))
    ax.scatter(df["slope_per_year"], df["cagr_pct"], s=20)
    ax.set_title("지표 추세: 연평균 기울기 vs CAGR(%)")
    ax.set_xlabel("slope_per_year")
    ax.set_ylabel("CAGR(%)")

    # 라벨링: |slope| 큰 상위 label_top개
    cand = df.reindex(df["slope_per_year"].abs().sort_values(ascending=False).index).head(label_top)
    for _, r in cand.iterrows():
        name = str(r.get("지표명", ""))
        ax.text(r["slope_per_year"], r["cagr_pct"], name[:16], fontsize=8, ha="left", va="bottom")
    _save(fig, "03_trends_scatter")

# 4) KPI 후보 막대(기울기 상위 순)
def plot_kpi_candidates(csv_path=OUT_DIR / "kpi_candidates.csv", topn=20):
    df = pd.read_csv(csv_path)
    if df.empty:
        print("[warn] KPI 후보가 비었습니다.")
        return
    df = df.sort_values("slope_per_year", ascending=False).head(topn)
    fig, ax = plt.subplots(figsize=(10, max(4, 0.35*len(df))))
    ax.barh(df["지표명"], df["slope_per_year"])
    ax.set_title(f"KPI 후보 Top {topn} (기울기 내림차순)")
    ax.set_xlabel("slope_per_year")
    ax.invert_yaxis()
    for y,(name,val) in enumerate(zip(df["지표명"], df["slope_per_year"])):
        ax.text(val, y, f"{val:.3g}", va="center", ha="left", fontsize=8)
    _save(fig, "04_kpi_candidates_slope")

# 5) 상관행렬 히트맵(상위 일부만 표시 가능)
def plot_corr_heatmap(csv_path=OUT_DIR / "correlation_pearson.csv", max_labels=40):
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt

    corr = pd.read_csv(csv_path, index_col=0)

    # 값 전부 숫자화(문자열 "nan" 등 처리), inf -> NaN
    corr = corr.apply(pd.to_numeric, errors="coerce").replace([np.inf, -np.inf], np.nan)

    # 비거나 전부 NaN이면 스킵
    if corr.empty or corr.to_numpy(dtype=float).size == 0 or np.all(np.isnan(corr.values)):
        print("[warn] 상관행렬이 비었거나 수치가 없습니다.")
        return

    # 라벨 개수 제한
    if corr.shape[0] > max_labels:
        corr = corr.iloc[:max_labels, :max_labels]

    fig, ax = plt.subplots(figsize=(8, 7))
    A = corr.to_numpy(dtype=float)
    im = ax.imshow(A, aspect="auto", vmin=-1, vmax=1)

    # 라벨 파싱: "(id, name, unit, src)" → name 우선
    def _short_label(s: str) -> str:
        s = str(s)
        if s.startswith("(") and "," in s and s.endswith(")"):
            parts = [p.strip() for p in s[1:-1].split(",")]
            if len(parts) >= 2:
                return parts[1][:12]
        return s[:12]

    ax.set_title("피어슨 상관 히트맵")
    ax.set_xticks(np.arange(corr.shape[1]))
    ax.set_yticks(np.arange(corr.shape[0]))
    ax.set_xticklabels([_short_label(i) for i in corr.columns], rotation=90, fontsize=7)
    ax.set_yticklabels([_short_label(i) for i in corr.index], fontsize=7)

    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label("r")

    _save(fig, "05_corr_heatmap")

# 6) 클러스터 그룹 크기 분포
def plot_cluster_sizes(csv_path=OUT_DIR / "clusters_corr0p7.csv"):
    df = pd.read_csv(csv_path)
    if df.empty:
        print("[warn] 클러스터 파일이 비었습니다.")
        return
    size_df = df.groupby("group_id")["지표명"].count().reset_index(name="size")
    size_df = size_df.sort_values("size", ascending=False)
    fig, ax = plt.subplots(figsize=(8,4))
    ax.bar(size_df["group_id"].astype(str), size_df["size"])
    ax.set_title("클러스터별 노드 수(|r|≥0.7 연결요소)")
    ax.set_xlabel("group_id")
    ax.set_ylabel("size")
    for i,(gid,sz) in enumerate(zip(size_df["group_id"], size_df["size"])):
        ax.text(i, sz, str(sz), ha="center", va="bottom", fontsize=8)
    _save(fig, "06_cluster_sizes")

# 7) 단일 지표 시계열 라인(임의 선택)
def plot_indicator_series(long_csv=OUT_DIR / "long_normalized.csv",
                          indicator_name:str=None, indicator_id=None):
    df = pd.read_csv(long_csv)
    # 필터 기준: 공개번호 우선, 없으면 지표명
    if indicator_id is not None and "공개번호" in df.columns:
        sdf = df[df["공개번호"] == indicator_id].copy()
        title = f"공개번호={indicator_id}"
    elif indicator_name is not None:
        sdf = df[df["지표명"] == indicator_name].copy()
        title = str(indicator_name)
    else:
        print("[info] 특정 지표 미지정 — 상위 1개 샘플 시각화")
        keycol = "공개번호" if "공개번호" in df.columns else "지표명"
        any_key = df[keycol].dropna().iloc[0]
        sdf = df[df[keycol]==any_key].copy()
        title = f"{keycol}={any_key}"

    sdf = sdf.dropna(subset=["year", "value_norm"])
    sdf = sdf.sort_values("year")
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(sdf["year"].astype(int), sdf["value_norm"])
    ax.set_title(f"지표 시계열: {title}")
    ax.set_xlabel("연도")
    ax.set_ylabel("값(value_norm)")
    for x, y in zip(sdf["year"].astype(int), sdf["value_norm"]):
        ax.text(x, y, f"{y:.2f}", fontsize=7, ha="center", va="bottom")
    _save(fig, f"07_series_{title}")

# 8) 이상치 지표 상위 N(횟수 기준)
def plot_top_outliers(csv_path=OUT_DIR / "outliers.csv", topn=20):
    df = pd.read_csv(csv_path)
    if df.empty:
        print("[warn] 이상치가 없습니다.")
        return
    keycol = "공개번호" if "공개번호" in df.columns else "지표명"
    cnt = df.groupby(["지표명", keycol])["outlier_flag"].sum().reset_index(name="n_outliers")
    cnt = cnt.sort_values("n_outliers", ascending=False).head(topn)
    fig, ax = plt.subplots(figsize=(10, max(4, 0.35*len(cnt))))
    ax.barh(cnt["지표명"], cnt["n_outliers"])
    ax.set_title(f"이상치 발생 상위 {topn} 지표")
    ax.set_xlabel("이상치 발생 횟수")
    ax.invert_yaxis()
    for y,(name,val) in enumerate(zip(cnt["지표명"], cnt["n_outliers"])):
        ax.text(val, y, str(int(val)), va="center", ha="left", fontsize=8)
    _save(fig, "08_top_outliers")


In [44]:
plot_missing_by_year()
plot_missing_by_indicator_topn(topn=20)
plot_kpi_candidates(topn=20)
plot_cluster_sizes()
plot_top_outliers(topn=20)

[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\01_missing_by_year.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\02_missing_by_indicator_topN.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\04_kpi_candidates_slope.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\05_corr_heatmap.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\06_cluster_sizes.png
[warn] 이상치가 없습니다.


In [46]:
plot_missing_by_year()

[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\01_missing_by_year.png


In [ ]:
plot_corr_heatmap(max_labels=40)

In [45]:
plot_top_outliers(topn=20)

[warn] 이상치가 없습니다.


In [40]:
plot_trends_scatter(label_top=12)

[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\03_trends_scatter.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\03_trends_scatter.png


#### 테마별 시계열 정보 종합

In [ ]:
# 개별 시계열 정보 생성
# import pandas as pd
# df = pd.read_csv(ROOT / "output" / "3. 도시정책지표" / "long_normalized.csv")
# for idx, index_name in enumerate(df['지표명'].unique()):
#     plot_indicator_series(indicator_name=index_name)
#     print(f'{idx}. 2010-2024년 {index_name} 변화 도표 생성 완료')

In [47]:
# ==== Filename / Plot Utils ===================================================
import re, math
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

FIG_DIR = Path(OUT_DIR) / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)

def sanitize_filename(s: str) -> str:
    # 윈도우 금지문자 / \ : * ? " < > | 제거 + 공백 정리
    s = re.sub(r'[\/\\\:\*\?\"\<\>\|]+', '_', str(s))
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def save_fig(fig, name: str):
    fname = sanitize_filename(name)
    p = FIG_DIR / f"{fname}.png"
    fig.tight_layout()
    fig.savefig(p, dpi=160, bbox_inches="tight")
    plt.close(fig)
    print(f"[saved] {p}")

# 개별 지표 라인 (기존 함수 대체)
def plot_indicator_series(long_csv=OUT_DIR / "long_normalized.csv",
                          indicator_name: str=None, indicator_id=None):
    df = pd.read_csv(long_csv)
    # 필터
    if indicator_id is not None and "공개번호" in df.columns:
        sdf = df[df["공개번호"] == indicator_id].copy()
        title = f"공개번호={indicator_id}"
    elif indicator_name is not None:
        sdf = df[df["지표명"] == indicator_name].copy()
        title = str(indicator_name)
    else:
        keycol = "공개번호" if "공개번호" in df.columns else "지표명"
        any_key = df[keycol].dropna().iloc[0]
        sdf = df[df[keycol]==any_key].copy()
        title = f"{keycol}={any_key}"

    if sdf.empty:
        print(f"[warn] 데이터 없음: {indicator_name or indicator_id}")
        return

    sdf = sdf.dropna(subset=["year"]).sort_values("year")
    ycol = "value_norm" if "value_norm" in sdf.columns else "value"
    sdf[ycol] = pd.to_numeric(sdf[ycol], errors="coerce")
    sdf = sdf.dropna(subset=[ycol])

    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(sdf["year"].astype(int), sdf[ycol])
    ax.set_title(f"지표 시계열: {title}")
    ax.set_xlabel("연도")
    ax.set_ylabel(ycol)
    save_fig(fig, f"07_series_{title}")

# ==== 테마별 그리드(서브플롯) =================================================
def plot_theme_grid_by_names(theme_name: str,
                             indicator_names: list,
                             long_csv=OUT_DIR / "long_normalized.csv",
                             ncols=3, height_per_row=2.6):
    """지표명 리스트를 받아 grid로 시각화"""
    df = pd.read_csv(long_csv)
    ycol = "value_norm" if "value_norm" in df.columns else "value"

    # 준비
    items = [n for n in indicator_names if n in set(df["지표명"])]
    if not items:
        print(f"[warn] 해당 지표 없음: {indicator_names[:3]} ...")
        return
    n = len(items)
    ncols = max(1, ncols)
    nrows = math.ceil(n / ncols)

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols,
                             figsize=(ncols*4, nrows*height_per_row),
                             squeeze=False)
    axes = axes.flatten()

    for i, name in enumerate(items):
        ax = axes[i]
        sdf = df[df["지표명"] == name].copy()
        sdf = sdf.dropna(subset=["year"])
        sdf[ycol] = pd.to_numeric(sdf[ycol], errors="coerce")
        sdf = sdf.dropna(subset=[ycol]).sort_values("year")
        if sdf.empty:
            ax.set_title(f"{name} (no data)"); ax.axis("off"); continue
        ax.plot(sdf["year"].astype(int), sdf[ycol])
        ax.set_title(str(name))
        ax.set_xlabel("연도"); ax.set_ylabel(ycol)

    # 남는 서브플롯 비우기
    for j in range(i+1, len(axes)):
        axes[j].axis("off")

    fig.suptitle(f"[테마] {theme_name}", y=0.99)
    save_fig(fig, f"10_theme_grid_{theme_name}")

def plot_theme_grid_by_cluster(group_id: int,
                               clusters_csv=OUT_DIR / "clusters_corr0p7.csv",
                               long_csv=OUT_DIR / "long_normalized.csv",
                               ncols=3, height_per_row=2.6, top_k=None):
    """corr 기반 cluster group_id를 grid로 시각화"""
    cdf = pd.read_csv(clusters_csv)
    if cdf.empty or "group_id" not in cdf.columns:
        print("[warn] 클러스터 파일 비어있음")
        return
    sub = cdf[cdf["group_id"] == group_id]
    if sub.empty:
        print(f"[warn] group_id={group_id} 없음")
        return
    names = list(sub["지표명"].dropna().unique())
    if top_k:
        names = names[:top_k]
    plot_theme_grid_by_names(f"Cluster {group_id}", names, long_csv, ncols, height_per_row)

# ==== 예시: 직접 테마 묶음 =====================================================
# 필요 시 본인 데이터에 맞게 수정
THEME_MAP = {
    "인구·가구": ["총인구","청년인구","고령인구","1인가구 수","1인가구 비율","평균 가구원 수"],
    "경제·일자리": ["고용률","실업률","기업수","소상공인사업체수","수출액","창업률","벤처기업수","여성기업 수"],
    "주거·부동산": ["주택보급률","주택가격","매매가격지수","전세가격지수","미분양주택수","주거환경만족도","노후주택비율","비주택거주가구비율"],
    "환경·에너지": ["초미세먼지 일평균 농도","초미세먼지 주의보 발령일수","1인당 온실가스 배출량",
                 "친환경 자동차 비율(보급율)","신재생에너지 보급률","1인당 생활폐기물 발생량","생활계폐기물 재활용률","1인당 공원 면적"],
    "교통·이동": ["대중교통 만족도","주차장확보율","지하철(경전철)","기차","택시","버스"],
    "건강·복지": ["주관적 건강수준 인지율","흡연율","건강생활 실천율","우울감","행복감","삶의 만족도","미충족 의료율"],
}

# ==== 실행 예시 ================================================================
# 1) 사전 정의 테마들 저장
for theme, names in THEME_MAP.items():
    plot_theme_grid_by_names(theme, names, ncols=3)

# 2) 클러스터 group_id별 저장(상위 몇 개만 예시)
# plot_theme_grid_by_cluster(group_id=1, top_k=12, ncols=3)
# plot_theme_grid_by_cluster(group_id=2, top_k=12, ncols=3)


[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\10_theme_grid_인구·가구.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\10_theme_grid_경제·일자리.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\10_theme_grid_주거·부동산.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\10_theme_grid_환경·에너지.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\10_theme_grid_교통·이동.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\10_theme_grid_건강·복지.png


#### 상관관계 분석 (히트맵)

In [49]:
# ==== Advanced correlation heatmap ============================================
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pathlib import Path

def _numeric_corr(csv_path):
    corr = pd.read_csv(csv_path, index_col=0)
    corr = corr.apply(pd.to_numeric, errors="coerce").replace([np.inf, -np.inf], np.nan)
    # 완전 결측 축 제거
    corr = corr.loc[corr.notna().any(axis=1), corr.notna().any(axis=0)]
    return corr

def _reorder_corr(corr: pd.DataFrame, method: str = "pca", n_clusters: int = 4):
    """
    Return: corr_reordered, order(list of labels), clusters(list of (start,end)) or None
    """
    labels = list(corr.index)
    clusters = None

    if method == "none" or corr.shape[0] <= 2:
        order = list(range(len(labels)))
        return corr.iloc[order, :].iloc[:, order], [labels[i] for i in order], clusters

    if method == "pca":
        # corr의 첫 고유벡터 기반 일차 정렬(스펙트럴 계열 간단 대체)
        try:
            w, v = np.linalg.eigh(corr.fillna(0).values)
            order = np.argsort(v[:, -1])  # 1st principal axis
        except Exception:
            order = np.argsort(corr.fillna(0).values.sum(1))
        ordered_labels = [labels[i] for i in order]
        return corr.loc[ordered_labels, ordered_labels], ordered_labels, clusters

    if method == "linkage":
        # 계층클러스터 정렬 + 블록 경계선 좌표
        try:
            from scipy.spatial.distance import squareform
            from scipy.cluster.hierarchy import linkage, leaves_list, fcluster
            # 상관→거리: d = sqrt(2*(1-r)), NaN은 1로 대체
            R = corr.fillna(0).clip(-1, 1).values
            D = np.sqrt(2*(1 - R))
            # linkage는 condensed 벡터 필요
            y = squareform(D, checks=False)
            Z = linkage(y, method="average")
            order = leaves_list(Z)
            ordered_labels = [labels[i] for i in order]
            corr_ord = corr.loc[ordered_labels, ordered_labels]
            # 군집 박스
            cl = fcluster(Z, t=n_clusters, criterion="maxclust")
            cl = cl[order]  # 정렬 순서에 맞춤
            # 각 클러스터의 연속 구간 추출
            clusters = []
            start = 0
            for i in range(1, len(cl)):
                if cl[i] != cl[i-1]:
                    clusters.append((start, i-1))
                    start = i
            clusters.append((start, len(cl)-1))
            return corr_ord, ordered_labels, clusters
        except ImportError:
            # SciPy 미설치 시 PCA로 대체
            return _reorder_corr(corr, method="pca", n_clusters=n_clusters)

    # fallback
    return _reorder_corr(corr, method="pca", n_clusters=n_clusters)

def plot_corr_heatmap_adv(csv_path=OUT_DIR / "correlation_pearson.csv",
                          max_labels=60,
                          method="linkage",     # 'none' | 'pca' | 'linkage'
                          n_clusters=4,
                          triangle="full",      # 'full' | 'lower' | 'upper'
                          annotate_if_small=True,
                          annotate_limit=30,
                          title="피어슨 상관 히트맵(고급)"):

    corr = _numeric_corr(csv_path)
    if corr.empty or corr.to_numpy(dtype=float).size == 0 or np.all(np.isnan(corr.values)):
        print("[warn] 상관행렬이 비었거나 수치가 없습니다.")
        return None

    # 라벨 개수 제한
    if corr.shape[0] > max_labels:
        corr = corr.iloc[:max_labels, :max_labels]

    # 재정렬
    corr_ord, labels, clusters = _reorder_corr(corr, method=method, n_clusters=n_clusters)

    # 삼각 마스킹
    A = corr_ord.to_numpy(dtype=float)
    m = A.copy()
    if triangle == "lower":
        m[np.triu_indices_from(m, k=1)] = np.nan
    elif triangle == "upper":
        m[np.tril_indices_from(m, k=-1)] = np.nan

    # 플롯
    fig, ax = plt.subplots(figsize=(min(10, 0.18*len(labels)+4), min(10, 0.18*len(labels)+4)))
    im = ax.imshow(m, aspect="auto", vmin=-1, vmax=1)
    ax.set_title(title + f"\n(method={method}, k={n_clusters})")
    ax.set_xticks(np.arange(len(labels)))
    ax.set_yticks(np.arange(len(labels)))

    def _short(s):
        s = str(s)
        # MultiIndex 문자열 처리: "(id, name, unit, src)" → name
        if s.startswith("(") and "," in s and s.endswith(")"):
            parts = [p.strip() for p in s[1:-1].split(",")]
            if len(parts) >= 2:
                return parts[1][:14]
        return s[:14]

    ax.set_xticklabels([_short(i) for i in labels], rotation=90, fontsize=7)
    ax.set_yticklabels([_short(i) for i in labels], fontsize=7)

    # 군집 블록 테두리
    if clusters:
        for (s, e) in clusters:
            # 사각형(패치 대신 선으로 테두리)
            ax.plot([s-0.5, e+0.5], [s-0.5, s-0.5], linewidth=1)
            ax.plot([s-0.5, e+0.5], [e+0.5, e+0.5], linewidth=1)
            ax.plot([s-0.5, s-0.5], [s-0.5, e+0.5], linewidth=1)
            ax.plot([e+0.5, e+0.5], [s-0.5, e+0.5], linewidth=1)

    # 주석: 소형 행렬일 때만(가독성)
    if annotate_if_small and len(labels) <= annotate_limit:
        for i in range(len(labels)):
            for j in range(len(labels)):
                val = A[i, j]
                if np.isnan(val):
                    continue
                ax.text(j, i, f"{val:.2f}", ha="center", va="center", fontsize=6)

    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label("r")

    _save(fig, f"05_corr_heatmap_adv_{method}_{triangle}")
    return {"labels": labels, "clusters": clusters}

# ==== Top-K correlated pairs (report용 테이블) ================================
def export_top_corr_pairs(csv_path=OUT_DIR / "correlation_pearson.csv",
                          out_csv=OUT_DIR / "top_corr_pairs.csv",
                          topk=100, min_abs_r=0.6):
    corr = _numeric_corr(csv_path)
    if corr.empty:
        print("[warn] 상관행렬이 비었습니다.")
        return
    # 스택 후 상삼각만 남기기(i<j), 자기상관 제외
    s = corr.stack(dropna=True).reset_index()
    s.columns = ["i", "j", "r"]
    s = s[s["i"] < s["j"]]
    s["abs_r"] = s["r"].abs()
    s = s.sort_values(["abs_r", "r"], ascending=[False, False])
    s = s[s["abs_r"] >= min_abs_r].head(topk)
    s.to_csv(out_csv, index=False, encoding="utf-8-sig")
    return s

# ==== 사용 예시 ================================================================
# 고급 히트맵: 계층군집 정렬 + 클러스터 박스 + 하삼각 표시
_ = plot_corr_heatmap_adv(method="linkage", n_clusters=6, triangle="lower",
                          max_labels=60, title="지표 상관 히트맵")

# 상관 상위쌍 100개 추출(절대값 기준)
_ = export_top_corr_pairs(topk=100, min_abs_r=0.7)


[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\05_corr_heatmap_adv_linkage_lower.png


C:\Users\wsm\AppData\Local\Temp\ipykernel_3020\339649808.py:147: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  s = corr.stack(dropna=True).reset_index()


#### 상관관계 분석 (Top 상관관계 쌍)

In [ ]:
# =========================================
# 효과적 시각화 세트 (Windows/Jupyter, matplotlib)
# =========================================
import math, re
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

FIG_DIR = Path(OUT_DIR) / "figures"
FIG_DIR.mkdir(parents=True, exist_ok=True)
matplotlib.rcParams["axes.unicode_minus"] = False
try:
    matplotlib.rcParams["font.family"] = "Malgun Gothic"
except Exception:
    pass

def _sanitize(s: str) -> str:
    s = re.sub(r'[\/\\\:\*\?\"\<\>\|]+', '_', str(s))
    return re.sub(r'\s+', ' ', s).strip()

def _save(fig, name: str, dpi=160):
    p = FIG_DIR / f"{_sanitize(name)}.png"
    fig.tight_layout()
    fig.savefig(p, dpi=dpi, bbox_inches="tight")
    plt.close(fig)
    print("[saved]", p)

# 1) KPI 스파크라인 그리드(상위 N개)
def plot_kpi_sparklines(kpi_csv=OUT_DIR/"kpi_candidates.csv",
                        long_csv=OUT_DIR/"long_normalized.csv",
                        topn=24, ncols=4):
    kpi = pd.read_csv(kpi_csv)
    if kpi.empty:
        print("[warn] KPI 후보 없음"); return
    names = kpi.sort_values("slope_per_year", ascending=False)["지표명"].head(topn).tolist()

    df = pd.read_csv(long_csv)
    ycol = "value_norm" if "value_norm" in df.columns else "value"
    df = df[df["지표명"].isin(names)]
    df["year"] = pd.to_numeric(df["year"], errors="coerce")

    n = len(names); ncols = max(1, ncols); nrows = math.ceil(n/ncols)
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*4, nrows*2.3), squeeze=False)
    axes = axes.flatten()

    for i, name in enumerate(names):
        ax = axes[i]
        s = df[df["지표명"]==name].dropna(subset=["year", ycol]).sort_values("year")
        if s.empty: ax.axis("off"); continue
        ax.plot(s["year"].astype(int), s[ycol], marker="o", ms=2)
        ax.set_title(str(name), fontsize=10)
        ax.set_xlabel("연도"); ax.set_ylabel(ycol); ax.grid(False)
    for j in range(i+1, len(axes)): axes[j].axis("off")
    fig.suptitle("KPI 후보 스파크라인(상위)", y=0.995)
    _save(fig, "11_kpi_sparklines_top")

# 2) 상관 상위쌍 가로 발산 바 차트(|r| 기준)
def plot_top_corr_diverging(pairs_csv=OUT_DIR/"top_corr_pairs.csv", topk=40):
    df = pd.read_csv(pairs_csv) if Path(pairs_csv).exists() else None
    if df is None or df.empty:
        print("[warn] top_corr_pairs.csv 없음"); return
    df = df.sort_values("r", ascending=True)
    if "abs_r" not in df.columns: df["abs_r"] = df["r"].abs()
    df = df.sort_values(["abs_r","r"], ascending=[False, False]).head(topk)
    # 라벨 축약 (i, j가 튜플 문자열일 수 있음)
    def _short(s):
        s = str(s)
        if s.startswith("(") and "," in s and s.endswith(")"):
            return s.split(",")[1].strip()[:18]
        return s[:18]
    df["pair"] = [f"{_short(i)} | {_short(j)}" for i,j in zip(df["i"], df["j"])]

    fig, ax = plt.subplots(figsize=(10, max(4, 0.35*len(df))))
    ax.barh(df["pair"], df["r"])  # 음수/양수 모두 표현
    ax.axvline(0, linewidth=1)
    ax.invert_yaxis()
    ax.set_title("상관 상위 쌍(|r| 기준, ±방향 표시)")
    ax.set_xlabel("r")
    for y,(name,val) in enumerate(zip(df["pair"], df["r"])):
        ax.text(val, y, f"{val:.2f}", va="center", ha="left" if val>=0 else "right", fontsize=8)
    _save(fig, "12_top_corr_pairs_diverging")

# 3) 결측 매트릭스(지표×연도) – 어디가 비는지 한눈에
def plot_missing_matrix(long_csv=OUT_DIR/"long.csv", max_indicators=60):
    df = pd.read_csv(long_csv)
    key = "공개번호" if "공개번호" in df.columns else "지표명"
    df["year"] = pd.to_numeric(df["year"], errors="coerce")
    df["is_valid"] = df["value"].notna() if "value" in df.columns else df["value_norm"].notna()
    # 피벗: 지표×연도
    mat = df.pivot_table(index="지표명", columns="year", values="is_valid", aggfunc="max").fillna(False)
    # 결측률 높은 순 정렬 후 상위 N만
    miss_ratio = 1 - mat.mean(1)
    mat = mat.loc[miss_ratio.sort_values(ascending=False).index]
    if mat.shape[0] > max_indicators: mat = mat.iloc[:max_indicators]
    fig, ax = plt.subplots(figsize=(10, max(4, 0.17*mat.shape[0])))
    ax.imshow(mat.values, aspect="auto", vmin=0, vmax=1)
    ax.set_title("결측 매트릭스(지표×연도) — 흰색:결측, 진한색:관측")
    ax.set_yticks(np.arange(mat.shape[0])); ax.set_yticklabels(mat.index, fontsize=7)
    ax.set_xticks(np.arange(mat.shape[1])); ax.set_xticklabels([int(c) for c in mat.columns], rotation=90, fontsize=7)
    _save(fig, "13_missing_matrix")

# 4) 클러스터별 소형 히트맵(상관행렬에서 group_id 기준)
def plot_cluster_heatmaps(clusters_csv=OUT_DIR/"clusters_corr0p7.csv",
                          corr_csv=OUT_DIR/"correlation_pearson.csv",
                          max_groups=8, max_per_group=30):
    if not Path(clusters_csv).exists() or not Path(corr_csv).exists():
        print("[warn] cluster/corr 파일 없음"); return
    corr = pd.read_csv(corr_csv, index_col=0).apply(pd.to_numeric, errors="coerce")
    cdf = pd.read_csv(clusters_csv)
    for gid in sorted(cdf["group_id"].unique())[:max_groups]:
        names = cdf[cdf["group_id"]==gid]["지표명"].dropna().unique().tolist()
        names = names[:max_per_group]
        cols = [c for c in corr.columns if any(str(n) in str(c) for n in names)]
        idxs = [i for i in corr.index   if any(str(n) in str(i) for n in names)]
        sub = corr.loc[idxs, cols]
        if sub.empty: continue
        # 숫자화 & NaN 제거
        sub = sub.apply(pd.to_numeric, errors="coerce")
        fig, ax = plt.subplots(figsize=(min(10, 0.22*sub.shape[1]+4), min(10, 0.22*sub.shape[0]+4)))
        ax.imshow(sub.values, aspect="auto", vmin=-1, vmax=1)
        ax.set_title(f"Cluster {gid} 히트맵")
        ax.set_xticks(np.arange(sub.shape[1])); ax.set_xticklabels([str(c)[:12] for c in sub.columns], rotation=90, fontsize=7)
        ax.set_yticks(np.arange(sub.shape[0])); ax.set_yticklabels([str(i)[:12] for i in sub.index], fontsize=7)
        _save(fig, f"14_cluster_{gid}_heatmap")

# 5) 이상치 강조 시계열(상위 N지표)
def plot_outlier_series(outliers_csv=OUT_DIR/"outliers.csv",
                        long_csv=OUT_DIR/"long_normalized.csv",
                        topn=12, ncols=3):
    if not Path(outliers_csv).exists(): print("[warn] outliers.csv 없음"); return
    out = pd.read_csv(outliers_csv)
    if out.empty: print("[warn] 이상치 없음"); return
    key = "공개번호" if "공개번호" in out.columns else "지표명"
    cnt = out.groupby(["지표명", key])["outlier_flag"].sum().reset_index(name="n_outliers")
    cnt = cnt.sort_values("n_outliers", ascending=False).head(topn)

    df = pd.read_csv(long_csv)
    ycol = "value_norm" if "value_norm" in df.columns else "value"
    n = len(cnt); ncols = max(1, ncols); nrows = math.ceil(n/ncols)
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols*4.3, nrows*2.5), squeeze=False)
    axes = axes.flatten()

    for i, (_, row) in enumerate(cnt.iterrows()):
        ax = axes[i]
        name = row["지표명"]
        s = df[df["지표명"]==name].dropna(subset=["year", ycol]).sort_values("year")
        ax.plot(s["year"].astype(int), s[ycol], marker="o", ms=3)
        # 이상치 위치만 마커로 한 번 더 찍기
        oo = out[(out["지표명"]==name) & (out["outlier_flag"]==True)]
        if not oo.empty:
            ax.plot(oo["year"].astype(int), oo["value_norm" if "value_norm" in oo.columns else "value"], linestyle="none", marker="x", ms=6)
        ax.set_title(f"{name} (outliers={int(row['n_outliers'])})", fontsize=10)
        ax.set_xlabel("연도"); ax.set_ylabel(ycol)
    for j in range(i+1, len(axes)): axes[j].axis("off")
    fig.suptitle("이상치 강조 시계열(상위)", y=0.995)
    _save(fig, "15_outlier_series_top")

# ------------ 실행 예시 ------------
plot_kpi_sparklines(topn=24, ncols=4)
plot_top_corr_diverging(topk=40)
plot_missing_matrix()
plot_cluster_heatmaps(max_groups=6, max_per_group=25)
plot_outlier_series(topn=12, ncols=3)


[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\11_kpi_sparklines_top.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\12_top_corr_pairs_diverging.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\13_missing_matrix.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\14_cluster_1_heatmap.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\14_cluster_2_heatmap.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\14_cluster_3_heatmap.png
[saved] d:\workspace\dacon_sri\output\3. 도시정책지표\figures\14_cluster_4_heatmap.png
[warn] 이상치 없음


: 